In [4]:
#결과 확인 및 출력
import pandas as pd
from konlpy.tag import Okt
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import DBSCAN
import numpy as np
import csv

df = pd.read_csv('article_crawling_with_company_name_and_thumbnails_221203.csv',names=['category','title','content','press','img','url'])
#중복 제거
df.drop_duplicates(['title'], ignore_index=True, inplace=True)
df.drop_duplicates(['content'], ignore_index=True, inplace=True)

#정치, 경제, 스포츠, 사회만 남기기
df = df[df['category'].isin(['정치','사회','경제','스포츠'])]

start = 1

okt = Okt()


drop_index_list = [] # 지워버릴 index를 담는 리스트
for i, row in df.iterrows():
    try:
        if len(row['content']) < 350: # 만약 350자 이하면 제거
            drop_index_list.append(i) # 지울 index 추가
    except:
        drop_index_list.append(i)

df.drop(drop_index_list, inplace=True) # 해당 index를 지우기

# index를 지우면 순회시 index 값이 중간중간 비기 때문에 index를 다시 지정
df.index = range(len(df))

#명사 추출
noun_list = []
for content in tqdm(df['content']):
    try:
        nouns = okt.nouns(content)
        noun_list.append(nouns)
    except Exception as e:
        print(e)
        print(content)
        noun_list.append([])
        continue

df['nouns'] = noun_list

drop_index_list = [] # 지워버릴 index를 담는 리스트
for i, row in df.iterrows():
    temp_nouns = row['nouns']
    if len(temp_nouns) == 0: # 만약 명사리스트가 비어 있다면
        drop_index_list.append(i) # 지울 index 추가

df.drop(drop_index_list, inplace=True) # 해당 index를 지우기

# index를 지우면 순회시 index 값이 중간중간 비기 때문에 index를 다시 지정
df.index = range(len(df))

# 문서를 명사 집합으로 보고 문서 리스트로 치환 (tfidfVectorizer 인풋 형태를 맞추기 위해)
text = [" ".join(noun) for noun in df['nouns']]

count_vectorizer = CountVectorizer(min_df = 2,ngram_range=(1,3))
count_vectorizer.fit(text)
vector = count_vectorizer.transform(text)

# vector = np.array(vector) # Normalizer를 이용해 변환된 벡터
model = DBSCAN(eps=0.63,min_samples=10, metric = "cosine")

result = model.fit_predict(vector)
df['result'] = result

df.drop(index=df[df['result']==-1].index, inplace=True)
df.index = range(len(df))
result = set(result)
result.remove(-1)


#결과 확인 용
for cluster_num in set(result):
    print("cluster num : {}".format(cluster_num))
    temp_df = df[df['result'] == cluster_num] # cluster num 별로 조회
    for i,row in temp_df.iterrows():
        print(row[1])
    print()
    start += 1

100%|██████████████████████████████████████████████████████████████████████████████| 3315/3315 [05:41<00:00,  9.70it/s]


cluster num : 0
서훈 심문, 10시간 만에 종료…‘박근혜 기록’ 깼다
서훈 심문 '역대 최장' 10시간 만에 종료···박근혜 기록 경신
'서해 피격' 서훈 前 국가안보실장, 구속심사 10시간 만에 종료
서훈 구속영장실질심사, 10시간 만에 종료...역대 최장 기록 경신
서훈 前국가안보실장, 영장실질심사 10시간 만에 종료(종합)
'서해피격' 서훈 역대 최장 영장심사…'박근혜 8시간40분' 경신
[단독] ‘서훈 공개 기자회견’이 증거인멸?…검찰, 구속영장서 주장
'서해피격' 공무원 친형 이래진씨 "문 전 대통령, 어디다 그런 망발을"
‘北 서해 피격 공무원’ 친형 이래진씨, 文 입장 발표에 분노 “누가 도를 넘고 있나”
이원석 총장 “서해 사건, 고인·유족 억울함 없게 처리할 것”
文 ‘최종 승인’ 입장문에 ‘서해 공무원’ 유족 “유죄 증거”
‘서해 피격’ 서훈 전 안보실장, 영장심사 출석… 구속 갈림길
서훈 전 실장 구속 기로...검찰 "영장 나면 文까지 수사 확대"
이원석 총장 "대부분 前정부부터 계속 진행해오던 수사"
이원석 검찰총장 “서해 피격 수사, 선입견 없어”
[머니S포토] 전해철 등 野 의원 "서욱 구속영장 청구 납득하기 어려워"
'서해피격' 서훈 영장심사 출석, 취재진 질문엔 "…"
이원석 "선입견·편견 없이 진실 밝히는 수사 진행"
'월북몰이' 의혹 서훈, 묵묵부답한 채 영장실질심사 출석 [뉴시스Pic]
'서해 공무원 피격' 서훈, 영장실질심사 출석… 질문엔 '묵묵부답'
[머니S포토] 법원 출석한 서훈 전 국가안보실장
이원석 검찰총장, 서해피격 수사에 "선입견도 편견도 없다"
[머니S포토] 월북몰이 주도 의혹, 구속기로에 놓인 '서훈'
서훈 구속기로…‘서해 피격’ 수사 성패 결정
[속보] '서해 피격' 서훈, 구속 심문 출석…'묵묵부답'
‘서해 피격’ 최종결정권자 서훈, 구속 심문 출석…‘묵묵부답’
'서해 피격' 유족 "文 말장난에 경악 금치 못해"
'서해 피격' 서훈 전 국가안보실장 오늘 구속영장 심사
文 ‘北 서해 공무원 피격’

전장연 출근길 지하철 시위 1년...시민 불편 극심, 예산 국회 반영은 성과
전장연 출근길 시위 '1년'... 예산 성과 냈지만 시민들 시선은 싸늘
"출근 좀 하자"…계속된 전장연 시위에 직장인도 불만 폭발
장애인 단체, 출근길 지하철 탑승 시위 재개
전장연 출근길 시위 재개 "1년 동안 지하철 시위했는데 尹 답 없어"
전장연, 이번엔 출근길 지하철 탑승시위…4호선 운행 지연
전장연 기습 시위에 아수라장 된 4호선…물리적 충돌도
전장연, 출근길 지하철 탔다 내렸다 '반복 시위'…4호선 운행 지연
지하철 파업 끝나니 전장연 또 기습시위…4·5호선 퇴근길 대혼잡
전장연, 퇴근길 지하철 탑승 시위…운행 일부 지연

cluster num : 13
방역당국, 겨울유행 정점 판단 '보류'‥"더 지켜봐야"
코로나 항체 얼마나 떨어졌나… 방역당국, 2차 항체 조사 실시
코로나19 겨울 유행 정점 지났나…방역 당국 “아직 몰라”
당국, 재유행 둔화세에도 “한파·실내활동 증가…긴장 계속”
항체양성률 1차 참여자 대상 추적조사 시작…"면역력 감소 본다"(종합)
힘못쓰는 재유행…당국 "백신접종·감염 증가 따른 면역력 증가 영향"
코로나19 항체양성률 2차 조사 착수…“면역력 감소 파악”
코로나 2차 항체조사, 오늘부터 시작… "이달 안으로 결과 분석"
2차 항체양성률 조사 착수…오늘부터 대상자 안내(종합)
항체양성률 1차 참여자에 2차조사 안내 시작…"면역력 감소 파악"
'면역 얼마나 떨어졌나'…코로나 항체 2차조사 들어간다
방역당국 "2주 후 감소세 전망 두고봐야"…한파·인플루엔자 고려

cluster num : 14
타임기술, 모트롤 '천궁 다기능레이더 창정비 요소개발'사업 수주 [경제소식]
재료연, 안전관리 우수연구실 인증…"안전문화 확산 위해 노력"
태양광·방산 이어 우주까지…김동관의 3번째 담대한 도전
한화그룹 우주사업 본궤도 박차… 한화에어로, 누리호 4회 발사 등 고도화 사업 수주
(영상)한화에어로, 韓 대표 우주기업 됐다...누리호 4회 추가 발사
“이제는 우주다”…

In [7]:
#결과 저장
import pandas as pd
from konlpy.tag import Okt
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import DBSCAN
import numpy as np
import csv

df = pd.read_csv('article_crawling_with_company_name_and_thumbnails_221203.csv',names=['category','title','content','press','img','url'])
#중복 제거
df.drop_duplicates(['title'], ignore_index=True, inplace=True)
df.drop_duplicates(['content'], ignore_index=True, inplace=True)

#정치, 경제, 스포츠, 사회만 남기기
df = df[df['category'].isin(['정치','사회','경제','스포츠'])]

start = 1

filename = 'clustered_article_221203_600.csv'
f = open(filename,'w',encoding='utf-8-sig', newline='')
writer = csv.writer(f)

okt = Okt()


drop_index_list = [] # 지워버릴 index를 담는 리스트
for i, row in df.iterrows():
    try:
        if len(row['content']) < 350: # 만약 350자 이하면 제거
            drop_index_list.append(i) # 지울 index 추가
    except:
        drop_index_list.append(i)

df.drop(drop_index_list, inplace=True) # 해당 index를 지우기

# index를 지우면 순회시 index 값이 중간중간 비기 때문에 index를 다시 지정
df.index = range(len(df))

#명사 추출
noun_list = []
for content in tqdm(df['content']):
    try:
        nouns = okt.nouns(content)
        noun_list.append(nouns)
    except Exception as e:
        print(e)
        print(content)
        noun_list.append([])
        continue

df['nouns'] = noun_list

drop_index_list = [] # 지워버릴 index를 담는 리스트
for i, row in df.iterrows():
    temp_nouns = row['nouns']
    if len(temp_nouns) == 0: # 만약 명사리스트가 비어 있다면
        drop_index_list.append(i) # 지울 index 추가

df.drop(drop_index_list, inplace=True) # 해당 index를 지우기

# index를 지우면 순회시 index 값이 중간중간 비기 때문에 index를 다시 지정
df.index = range(len(df))

# 문서를 명사 집합으로 보고 문서 리스트로 치환 (tfidfVectorizer 인풋 형태를 맞추기 위해)
text = [" ".join(noun) for noun in df['nouns']]

count_vectorizer = CountVectorizer(ngram_range=(1,4))
count_vectorizer.fit(text)
vector = count_vectorizer.transform(text)

# vector = np.array(vector) # Normalizer를 이용해 변환된 벡터
model = DBSCAN(eps=0.6,min_samples=10, metric = "cosine")

result = model.fit_predict(vector)
df['result'] = result

df.drop(index=df[df['result']==-1].index, inplace=True)
df.index = range(len(df))
result = set(result)
result.remove(-1)


#결과 확인 용
for cluster_num in set(result):
    print("cluster num : {}".format(cluster_num))
    temp_df = df[df['result'] == cluster_num] # cluster num 별로 조회
    for i,row in temp_df.iterrows():
        print(row[1])
        writer.writerow([start,row['category'],row['title'],row['content'],row['press'],row['img'],row['url']])
    print()
    start += 1
f.close()

100%|██████████████████████████████████████████████████████████████████████████████| 1470/1470 [01:56<00:00, 12.57it/s]


cluster num : 0
尹, 벤투 감독·손흥민과 통화…"국민에 큰 선물"
尹대통령, 벤투·손흥민과 통화 “16강 큰 선물 고맙다”
尹, 벤투·손흥민과 통화 "국민들 힘든데 위로와 희망 줘 고맙다"
윤 대통령, 벤투에 "리더십으로 좋은 결과", 손흥민에 "얼굴 괜찮냐"
尹 "국민에 위로와 희망, 기쁨"...벤투 감독·손흥민과 통화
손흥민, 尹대통령과 통화…"나라 위해 한 몸 바치겠단 생각"
尹대통령 "이런 큰 선물을…", 벤투 감독 "국민 위해 최선 노력"
윤 대통령, 벤투·손흥민과 통화…“어려운 국가 상황에 기쁨 줘 감사”
윤 대통령, 벤투 감독·손흥민 선수와 통화 “위로와 희망, 기쁨 줘 고맙다”
尹대통령, ‘월드컵 16강 진출’ 벤투 감독·손흥민과 통화
“얼굴 부상은?” “나라 위해 바치겠다”  … 尹 대통령, 손흥민 직접 전화 격려 [2022 카타르]

cluster num : 1
화물연대 "죽이려면 죽여라, 어차피 이렇게는 못산다"
민주노총 분노 폭발 "정부가 포기해버린 국민 안전, 화물연대가 끝까지 지킬 것"
'촛불집회 참석' 화물노동자 "'경제동맥' 역할…尹 정권에 무시당해"
서울·부산서 민주노총 대규모 집회…”화물연대 파업에 연대”
화물연대 파업 열흘째…부산신항서 영남권 노동자대회
화물연대 파업 열흘째…민주노총, 여의도서 전국노동자대회
"화물연대 탄압은 노동계 탄압"…부산신항서 민주노총 노동자대회 열려
민주노총, 화물연대 파업 거점 부산항 신항서 영남권 노동자대회
"화물연대 투쟁 사수하자"···민주노총 서울·부산서 대규모 집회
"노란봉투법·화물연대 승리"…민노총 부산신항서 노동자대회 5000명 집결
화물연대 총파업 10일째… 전국 물동량 회복세
시멘트 업무명령 조사 완료…차주 175명 운송 재개 의사 밝혀
‘화물연대 총파업 지지’ ‘노조법 개정 촉구’...민주노총 서울·부산 노동자대회
항만 물동량 평시 63% 회복…민주노총, 반발 집회
항만 물동량 평시 63%…민주노총, 업무개시명령 철회 촉구(종합)
민주노총, 화물연대 파업 거점 부산항에서 